<a href="https://colab.research.google.com/github/aswinaus/Assignments/blob/main/Agent_RewardFunction_TextOverlap_GroundTruth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-index -q
!pip install langchain -q
!pip install langchain_experimental -q

In [ ]:
import os
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from google.colab import userdata
# Set the OpenAI API key as an environment variable
os.environ["OPENAI_API_KEY"] =  userdata.get('OPENAI_API_KEY')

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
# Setup OpenAI Model and Embeddings used for indexing the documents
Settings.llm = OpenAI(model='gpt-4o-mini', temperature=0.2)
Settings.embed_model = OpenAIEmbedding(model='text-embedding-3-small')
Settings.chunk_size = 1024

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_dir = '/content/drive/MyDrive' # Input a data dir path from your mounted Google Drive

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector
from llama_index.core import SimpleDirectoryReader
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core import VectorStoreIndex, SummaryIndex

In [ ]:
# In order to avoid repeated calls to LLMs we can store the documents index and load it if present else create it
PERSIST_INDEX_DIR = f"/{data_dir}/RAG/data/"
def get_index(index_name, doc_file_path):
  index = None
  if not os.path.exists(f"{PERSIST_INDEX_DIR}{index_name}/"):
    # Load the documents
    documents = SimpleDirectoryReader(input_files=[doc_file_path]).load_data()
    index = VectorStoreIndex.from_documents(documents)
    # Store the index to disk
    index.storage_context.persist(f"{PERSIST_INDEX_DIR}{index_name}/")
  else: # Load index from disk
    storage_context = StorageContext.from_defaults(persist_dir=f"{PERSIST_INDEX_DIR}{index_name}/")
    index = load_index_from_storage(storage_context)

  return index

In [ ]:
# Load OECD guidelines documents for Transfer Pricing
docs_OECD_guidelines = SimpleDirectoryReader(f"{data_dir}/RAG/data/OECD/").load_data()
# Load OECD guidelines documents for Form990
docs_Form990_guidelines = SimpleDirectoryReader(f"{data_dir}/RAG/data/Form990/").load_data()

In [ ]:
#initialise a storage context and use that for both Vector Index and Summary Index for OECD
#split the OECD document into multiple nodes
oecd_nodes = Settings.node_parser.get_nodes_from_documents(docs_OECD_guidelines)
#split the Form990 document into multiple nodes
form990_nodes = Settings.node_parser.get_nodes_from_documents(docs_Form990_guidelines)

storage_context = StorageContext.from_defaults()

storage_context.docstore.add_documents(oecd_nodes)
storage_context.docstore.add_documents(form990_nodes)
# Setup Vector and Summary Index from Storage Context
summary_index = SummaryIndex(oecd_nodes, storage_context=storage_context)
vector_index = VectorStoreIndex(oecd_nodes, storage_context=storage_context)

# Setup Indices.In order to avoid repeated calls to LLMs we can store the documents index and load it if present else create it
OECD_index = get_index("OECDTPGuidelines",f"{data_dir}/RAG/data/OECD/OECD_Transfer_Pricing_Guidelines.pdf")
form990_guidelines_index = get_index("Form990Guidelines",f"{data_dir}/RAG/data/Form990/Form990_Guidelines.pdf")

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

# Create the query engines
OECD_engine = OECD_index.as_query_engine(similarity_top_k=3)
form990_guidelines_engine = form990_guidelines_index.as_query_engine(similarity_top_k=3)
# Create tools for the query engines
OECD_query_tool = QueryEngineTool(
                      query_engine=OECD_engine,
                      metadata=ToolMetadata(
                          name="OECD_QueryEngineTool_2022",
                          description="Provides information about Transfer Pricing Guidelines for Organization from OECD for year 2022"
                      )
                    )

Form990_query_tool = QueryEngineTool(
                      query_engine=form990_guidelines_engine,
                      metadata=ToolMetadata(
                          name="form990_2022",
                          description="Provides information about Form990 filling guidelines for Non-Profit Organization only from the index which was set for Form990_Guidelines.pdf "
                      )
                    )

tools = [OECD_query_tool, Form990_query_tool]

filing_engine = RouterQueryEngine(
                      selector= LLMSingleSelector.from_defaults(),
                      query_engine_tools=tools
                      )

In [ ]:
#Agentic Router RAG -
from llama_index.agent.openai import OpenAIAgent
agent = OpenAIAgent.from_tools(tools=tools, verbose=True)
# Uncomment and use the below call for interactive session
#agent.chat_repl()
response = agent.chat("What is Form990 EZ and when should an organiaztion complete Form990 EZ form? And how is it different from Schedule H? Can you show the results in side by side comparison table with headers and also link to the document?")
print (response)

In [ ]:
from llama_index.agent.openai import OpenAIAssistantAgent
agent = OpenAIAssistantAgent.from_new(
          name = "OECD and Form990 Agent",
          instructions= "You are an assistant that provides answers to questions on OECD and Form990. And make sure the answers are retreived form the OECD and Form990 pdf's only. No data from open Internet. Whenever there is comparison make sure the results are in side by side comparison table with headers and add links to the document.",
          tools=tools,
          verbose=True,
          run_retrieve_sleep_time=1.0
        )
response = agent.chat("What does Articles 9 and 25 of the OECD Model Tax Convention state?")
print (response)

In [ ]:
questions = ["What does Articles 9 of the OECD Model Tax Convention state?",
             "What does Articles 25 of the OECD Model Tax Convention state?"]
ground_truth = ["addresses corresponding adjustments in transfer pricing",
                "outlines the mutual agreement procedure, which resolves disputes related to the application of double tax conventions."]

In [ ]:
!pip install datasets --quiet
from datasets import Dataset

In [ ]:
import re
from collections import Counter

def jaccard_similarity(text1, text2):
    """Calculates the Jaccard similarity between two texts."""
    # Tokenize and convert to lowercase
    tokens1 = set(re.findall(r'\w+', text1.lower()))
    tokens2 = set(re.findall(r'\w+', text2.lower()))

    # Handle empty sets
    if not tokens1 and not tokens2:
        return 1.0
    if not tokens1 or not tokens2:
        return 0.0

    intersection = len(tokens1.intersection(tokens2))
    union = len(tokens1.union(tokens2))
    return intersection / union

def text_overlap_reward(retrieved_context, ground_truth):
    """
    Calculates a reward based on text overlap similarity (Jaccard index)
    between the retrieved context and the ground truth.

    Args:
        retrieved_context (str): The text from the retrieved documents.
        ground_truth (str): The ground truth text.

    Returns:
        float: A score between 0 and 1 representing the text overlap similarity.
    """
    return jaccard_similarity(retrieved_context, ground_truth)

# Example usage:
# Assuming 'contexts' is a list of strings from your retrieved documents
# and 'ground_truth_text' is your ground truth string.
# For a single question and its retrieved contexts:
# combined_context = " ".join(contexts[0]) # Combine retrieved contexts into a single string
# reward = text_overlap_reward(combined_context, ground_truth[0])
# print(f"Text Overlap Reward: {reward}")

In [ ]:
answers  = []
contexts = []
text_overlap_rewards = [] # List to store text overlap rewards


# traversing each question and passing into the chain to get answer from the system
# Define the retriever from the OECD index
retriever = OECD_index.as_retriever()

for i, question in enumerate(questions):
    response = agent.chat(question)
    answers.append(response.response) # Extract the string response
    retrieved_docs = retriever.retrieve(question)
    context_texts = [docs.node.text for docs in retrieved_docs]
    contexts.append(context_texts)

    # Calculate text overlap reward
    # Combine retrieved contexts into a single string for similarity calculation
    combined_context = " ".join(context_texts)
    text_overlap_reward_score = text_overlap_reward(combined_context, ground_truth[i])
    text_overlap_rewards.append(text_overlap_reward_score)


# Preparing the dataset
data = {
    "question": questions,
    "answer": answers,
    "ground_truth": ground_truth,
    "contexts": contexts, # Add the contexts to the dataset
    "text_overlap_reward": text_overlap_rewards, # Add the text overlap rewards
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

dataset.to_pandas()

In [ ]:
!pip install ragas --quiet
import ragas

In [ ]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset=dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

df = result.to_pandas()
df

In [ ]:
#External API to showcase function calling
from llama_index.core.tools import FunctionTool
import requests
from requests.auth import HTTPDigestAuth
import json

def call_form990API(param):
  url = "https://projects.propublica.org/nonprofits/api/v2/search.json?q="+param
  apiResponse = requests.get(url, verify=True)
  OrganizationData = json.loads(apiResponse.content)
  return OrganizationData

OrganizationData=call_form990API("north")
json_formatted_str = json.dumps(OrganizationData, indent=4)
print(json_formatted_str)

form990_function_tool = FunctionTool.from_defaults(fn=call_form990API)
#tools = [call_form990API]
# Create the Agent with our tools
#agent = OpenAIAgent.from_tools(tools, verbose=True)
#response = agent.query("North")

In [ ]:
#Reasoning and Act Agent
from llama_index.core.agent import ReActAgent
query_engine_tools = [OECD_query_tool, Form990_query_tool, form990_function_tool]
agent = ReActAgent.from_tools(
            tools= query_engine_tools,
            verbose=True,
            context="""You are AI Tax Assistant. You will guide tax professionals for filling Form990 and answer queries related to Transfer Pricing based on the OECD guidelines.
                      And make sure the answers are retreived form the OECD and Form990 pdf's only. No data from open Internet.
                      Whenever there is comparison make sure the results are in side by side comparison table with headers and add links to the document."""
          )
response = agent.query("Please compare and analyse Form990 Tax reporting process and Transfer Pricing methodologies used in identifying Intangibles used within Multinational Firms? If the analysis determines these process are for two different sectors then call the Form990 API with param north and include the results as part of the response?")
print (response)

In [ ]:
#One shot Query Planning
from llama_index.core.query_engine import SubQuestionQueryEngine
sub_question_query = "Compare the Form990 Tax reporting process for Non Profit Organizations and Transfer Pricing methodologies used in identifying Intangibles used within a Multinational Firms?"
query_planning_engine = SubQuestionQueryEngine.from_defaults(
                          query_engine_tools=tools,
                          use_async=True
                        )
response = query_planning_engine.query(sub_question_query)
print (response)